# --- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..
...##.
.####.
....#.
s###..
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [ ]:
import logging
import sys

sys.path.append("..")
from utilsp import get_logger, get_data, timer, test

logger = get_logger("day09", logging.INFO)
data = get_data("day09")

logger.info("initialization complete!")

## Solution Part One

In [ ]:
from types import SimpleNamespace
from IPython.display import clear_output
import time


class Simulation(object):
    """Base class which creates an iterable object which can be rendered"""

    def __init__(self, steps, plot_x: int = 50, plot_y: int = 15):
        self.steps = steps
        self.step_n = 0
        self.frame_size = [
            -int(plot_x / 2),
            int(plot_x / 2),
            -int(plot_y / 2),
            int(plot_y / 2),
        ]

    def frame(self):
        """Returns a frame of the simulation"""
        raise NotImplementedError

    def step(self):
        """Advances the simulation by one step"""
        self.step_n += 1

    def __repr__(self):
        matrix = self.frame()
        frame = [
            "".join(
                [matrix[x, y] for x in range(self.frame_size[0], self.frame_size[1])]
            )
            for y in range(self.frame_size[2], self.frame_size[3])
        ]
        frame.reverse()
        return "\n".join(frame) + "\n"

    def __iter__(self):
        return self

    def __next__(self):
        if self.step_n < self.steps:
            self.step()
            return self
        else:
            raise StopIteration


class Rope(Simulation):
    def __init__(self, motions: list, plot_x: int = 50, plot_y: int = 15):
        self.motions = [m for l in motions for m in self._parse_line(l)]
        super().__init__(len(self.motions), plot_x, plot_y)
        self.head = SimpleNamespace(name="H", x=0, y=0)
        self.tail = SimpleNamespace(name="T", x=0, y=0)
        self.visited = {(0, 0)}

    @staticmethod
    def _parse_line(line) -> list:
        direction, repeat = line.split(" ")
        abbrevs = {"L": (-1, 0), "R": (1, 0), "U": (0, 1), "D": (0, -1)}
        dir_x, dir_y = abbrevs[direction]
        return [(dir_x, dir_y) for _ in range(int(repeat))]

    def step(self):
        self.head.x += self.motions[self.step_n][0]
        self.head.y += self.motions[self.step_n][1]
        self._update_tail()
        self.step_n += 1

    def frame(self):
        matrix = {
            (x, y): "."
            for x in range(self.frame_size[0], self.frame_size[1])
            for y in range(self.frame_size[2], self.frame_size[3])
        }
        for x, y in self.visited:
            matrix[x, y] = "#"
        matrix[self.tail.x, self.tail.y] = self.tail.name
        matrix[self.head.x, self.head.y] = self.head.name
        return matrix

    @staticmethod
    def _update_knot(head, tail) -> ((), ()):
        """Returns the new head and tail positions"""
        moving_right = True if head.x > tail.x else False
        moving_up = True if head.y > tail.y else False
        dx = abs(head.x - tail.x)
        dy = abs(head.y - tail.y)
        logger.debug(f"dx {dx}, dy {dy}")
        if dx == 2 and dy == 0:  # moving horizontally
            tail.x += 1 if moving_right else -1
        elif dy == 2 and dx == 0:  # moving vertically
            tail.y += 1 if moving_up else -1
        elif dx + dy > 2:  # diagonal
            tail.y += 1 if moving_up else -1
            tail.x += 1 if moving_right else -1
        else:
            pass
        return head, tail

    def _update_tail(self):
        """Updates the tail position"""
        head, tail = self.head, self.tail
        self.head, self.tail = self._update_knot(head, tail)
        self.visited.add((self.tail.x, self.tail.y))


@timer
def part1(input_: str) -> int:
    simulation = Rope(input_.split("\n"))
    for rope in simulation:
        clear_output(wait=True)
        print(rope)
        time.sleep(0.1)

    return len(simulation.visited)


if __name__ == "__main__":
    test(func=part1, input_=data.example_input, expected=data.test_answer1)
    print("waiting 10 seconds for next run...")
    time.sleep(10)
    print("solution part 1:", part1(data.input))

# --- Part Two ---

## Solution Part Two

In [ ]:
class RopeExtension(Rope):  # double inheritance, sue me
    """This class extends the rope class, get it?"""

    def __init__(self, motions: list, knots: int):
        super().__init__(motions)
        self.knots = [
            SimpleNamespace(name=str(i), x=0, y=0) for i in range(1, knots + 1)
        ]

    def _update_tail(self):
        # update head
        self.head, self.knots[0] = self._update_knot(self.head, self.knots[0])

        # update knots
        head, knot = self.head, self.knots[0]
        for i in range(1, len(self.knots)):
            knot, self.knots[i] = self._update_knot(knot, self.knots[i])
            knot = self.knots[i]
        self.visited.add((self.knots[-1].x, self.knots[-1].y))

    def frame(self):
        matrix = {
            (x, y): "."
            for x in range(self.frame_size[0], self.frame_size[1])
            for y in range(self.frame_size[2], self.frame_size[3])
        }
        for x, y in self.visited:
            matrix[x, y] = "#"
        for knot in self.knots:
            matrix[knot.x, knot.y] = knot.name
        matrix[self.head.x, self.head.y] = self.head.name
        return matrix


@timer
def part2(input_: str) -> int:
    simulation = RopeExtension(input_.split("\n"), knots=9)
    for rope in simulation:
        clear_output(wait=True)
        print(rope)
        time.sleep(0.1)
    return len(simulation.visited)


if __name__ == "__main__":
    test(func=part2, input_=data.example_input, expected=data.test_answer2)
    print("waiting 10 seconds for next run...")
    time.sleep(10)
    print("solution part 2:", part2(data.input))

# Performance tuning
Unsuprisingly, the code is much faster without plotting and delays.

In [ ]:
@timer
def part1(input_: str) -> int:
    simulation = Rope(input_.split("\n"))
    *_, last = simulation
    return len(last.visited)


test(func=part1, input_=data.example_input, expected=data.test_answer1)

In [ ]:
@timer
def part2(input_: str) -> int:
    simulation = RopeExtension(input_.split("\n"), knots=9)
    *_, last = simulation
    return len(last.visited)


test(func=part2, input_=data.example_input, expected=data.test_answer2)